<a href="https://www.kaggle.com/code/mcsadri/linear-regresssion?scriptVersionId=127868104" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Linear Regression with sklearn

**Author**: [Manuch S.](https://github.com/mcsadri)  
**Input**: [2,2k+ Scotch Whisky Reviews](https://www.kaggle.com/datasets/koki25ando/22000-scotch-whisky-reviews): a dataset including 2,2k+ Scotch Whisky reviews (originally collected from the Whisky Advocate database).

#### Overview

Performing exploratory analysis by using Linear Regressions on a Kaggle data set.

#### Feature Tasks and Requirements

- [X] Select a Kaggle data set that is suitable for Linear Regression.
    - Note make sure the data set is stored as csv file/s.
    - Data set must be continous values appropriate for a Linear Regression. If you’re not sure then ask Instructor or TA.
- [X] Load the data you receive into a Pandas DataFrame.
- [X] Show the first five rows of the data set.
- [X] Show the description and the info of the data set.
- [X] Ensure that any date columns have been cast into a datetime object in your DataFrame.
- [ ] Using a regression model, split your data into train and test portions.
- [ ] Fit your training split to the regression model.
- [ ] Show your regression model’s score.
- [ ] Draw at least three conclusions from your regression model.
- [ ] Your notebook should be clutter free and polished.

#### UAT

No acceptance tests today, but Notebook should clearly display its conclusions.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import re
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/22000-scotch-whisky-reviews/scotch_review.csv


### Read the original dataset, and display the first 5 rows

In [2]:
raw_df = pd.read_csv("/kaggle/input/22000-scotch-whisky-reviews/scotch_review.csv")
raw_df.head(5)

,Unnamed: 0,name,category,review.point,price,currency,description
0,1,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,$,"Magnificently powerful and intense. Caramels, ..."
1,2,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,$,What impresses me most is how this whisky evol...
2,3,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,$,There have been some legendary Bowmores from t...
3,4,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,$,With a name inspired by a 1926 Buster Keaton m...
4,5,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,$,"Captivating, enticing, and wonderfully charmin..."


### get describe(), info(), and shape() for the raw dataset

In [3]:
print("The description of the raw dataset is:")
print("")
print(raw_df.describe())
print("")
print("The raw dataset information is:")
print("")
raw_df.info()
print("")
print(f"The raw dataset shape is: {np.shape(raw_df)}")
print("")

The description of the raw dataset is:

        Unnamed: 0  review.point
count  2247.000000   2247.000000
mean   1124.000000     86.700045
std     648.797349      4.054055
min       1.000000     63.000000
25%     562.500000     84.000000
50%    1124.000000     87.000000
75%    1685.500000     90.000000
max    2247.000000     97.000000

The raw dataset information is:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2247 entries, 0 to 2246
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    2247 non-null   int64 
 1   name          2247 non-null   object
 2   category      2247 non-null   object
 3   review.point  2247 non-null   int64 
 4   price         2247 non-null   object
 5   currency      2247 non-null   object
 6   description   2247 non-null   object
dtypes: int64(2), object(5)
memory usage: 123.0+ KB

The raw dataset shape is: (2247, 7)



### Validate which currencies are present in the DataFrame (so we can scrub any non-$ later, if needed)

In [4]:
currency_types = raw_df['currency'].value_counts().sort_values(ascending=False)
currency_types

$    2247
Name: currency, dtype: int64

### Using the review.point and price columns: Clean price data so it can be converted to int, and remove index 19 from both as an incompatible data point. Create new DataFrame with these two series. 

### display the first 5 rows of the modified dataframe

In [5]:
# str.replace and regex pattern solution via https://www.kaggle.com/code/theenduser/scotch-whiskey-data-cleaning?scriptVersionId=5690584&cellId=13
series_price = raw_df.price.str.replace("[({',$/qwertyuioplkjhgfdsazxcvbnm%:]", "", regex=True)
series_price = series_price.drop(19)
series_price = series_price.astype(float).astype(int)

series_review = raw_df['review.point']
series_review = series_review.drop(19)

df = pd.concat([series_review, series_price], axis = 1)

df.head(5)

,review.point,price
0,97,225
1,97,4500
2,97,13500
3,96,325
4,96,160


### get describe(), info(), and shape() for the modified dataset

In [6]:
print("The description of the dataset is:")
print("")
print(df.describe())
print("")
print("The dataset's information is:")
print("")
df.info()
print("")
print(f"The dataset shape is: {np.shape(df)}")
print("")

The description of the dataset is:

       review.point          price
count   2246.000000    2246.000000
mean      86.696349     636.725289
std        4.051171    4693.904590
min       63.000000      12.000000
25%       84.000000      70.000000
50%       87.000000     110.000000
75%       90.000000     200.000000
max       97.000000  157000.000000

The dataset's information is:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2246 entries, 0 to 2246
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   review.point  2246 non-null   int64
 1   price         2246 non-null   int64
dtypes: int64(2)
memory usage: 52.6 KB

The dataset shape is: (2246, 2)

